In [106]:
from glob import glob
import itertools
import os.path
import re
import tarfile
import time
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams

from html.parser import HTMLParser
from urllib.request import urlretrieve
from sklearn.datasets import get_data_home
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB

In [107]:
import spacy
import en_core_web_sm
import re
from nltk.tokenize.regexp import RegexpTokenizer

In [108]:
df = pd.read_csv('../SmartEmailTracker/emaildataset.csv', usecols = ['Subject','Body', 'Class'])
print(df.shape)
df.head()

(951, 3)


,Subject,Body,Class
0,Transaction no. 072558 is unresolved.,Sorry to inform that there has been only a par...,Pending
1,Order for new Cheque book,"Good morning, I want to place an order for an ...",General
2,Required money acquired. Transaction 847047 is...,Hello! This is to inform you that I have recei...,Processing
3,Asking for the details for transaction 746078,I request you to kindly send the status of my ...,Request
4,Partial payment for transaction 535918,Hello!! Greetings for the day. Status of trans...,Pending


In [109]:
true_k = df.Class.unique().shape[0]

In [110]:
true_k

6

In [111]:
nlp_ = en_core_web_sm.load()

In [112]:
nlp = spacy.load('en')

In [113]:
my_stop = {"'d", "'ll", "'m", "'re", "'s", "'ve",'a','cc','subject','http', 'gbp', 'usd', 'eur', 'inr', 'cad', 'thanks', "acc", "id", 'account', 'regards', 'hi', 'hello', 'thank you', 'greetings', 'about','above', 'across','after','afterwards','against','alone','along','already','also','although','always','am','among', 'amongst','amount','an','and','another','any','anyhow','anyone','anything','anyway','anywhere','are','around','as', 'at','back','be','became','because','become','becomes','becoming','been','before','beforehand','behind','being','below', 'beside','besides','between','beyond','both','bottom','but','by','ca','call','can','could','did', 'do', 'does', 'doing', 'done', 'down', 'due', 'during', 'each', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had', 'has', 'have', 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'it', 'its', 'itself', 'just', 'keep', 'last', 'latter', 'latterly', 'least', 'less', 'made', 'make', 'many', 'may', 'me', 'meanwhile', 'might', 'mine', 'more', 'moreover', 'most', 'mostly', 'move', 'much', 'must', 'my', 'myself', 'name', 'namely', 'neither', 'never', 'nevertheless', 'next', 'nine', 'no', 'nobody', 'now', 'nowhere', 'of', 'off', 'often', 'on', 'once', 'one', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'part', 'per', 'perhaps', 'please', 'put', 'quite', 'rather', 're', 'really', 'regarding', 'same', 'say', 'see', 'seem', 'seemed', 'seeming', 'seems', 'serious', 'several', 'she', 'should', 'show', 'side', 'since', 'six', 'sixty', 'so', 'some', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhere', 'still', 'such', 'take', 'ten', 'than', 'that', 'the', 'their', 'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'third', 'this', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'top', 'toward', 'towards', 'twelve', 'twenty', 'two', 'under', 'unless', 'until', 'up', 'upon', 'us', 'used', 'using', 'various', 'very', 'via', 'was', 'we', 'well', 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with', 'within', 'would', 'yet', 'you', 'your', 'yours', 'yourself', 'yourselves', '‘d', '‘ll', '‘m', '‘re', '‘s', '‘ve', '’d', '’ll', '’m', '’re', '’s', '’ve'}

In [114]:
def clean(text):
    
    text=text.rstrip()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = " ".join([i for i in text.lower().split()])
    print(text)
    
#     customize_stop_words = ["cc","subject","http", "gbp", "usd", "eur", "inr", "cad","thanks", "acc", "id", "account", "regards", "hi", "hello", "thank you", "greetings"]
#     rem_stop = ["not", "wasn't", "hadn't", "won't", "can't", "didn't"]
    
#     for w in customize_stop_words:
#         nlp.vocab[w].is_stop = True
    
#     for w in rem_stop:
#         nlp.vocab[w].is_stop = False
    
    doc = nlp(text)
    
    normalized = " ".join(token.lemma_ token in doc if token not in my_stop)
    
    doc = " ".join(token.orth_ for token in nlp(normalized) if not token.is_punct | token.is_space)
    return doc

In [115]:
print(clean("payment in id 1234 for amount 14859 GBP has been freezed"))

payment in id for amount gbp has been freezed
payment would freeze


In [116]:
for i in range(df.shape[0]):
    # merge subject and body strings
    df['Text_Data'] = (df['Subject'] + " " + df['Body'])

In [117]:
def converter(x):
    try:
        return ' '.join([x.lower() for x in str(x).split()])
    except AttributeError:
        return None  # or some other value

df['Text_Data'] = df['Text_Data'].apply(converter)

In [118]:
text_clean=[]

for i in range(df.shape[0]):
    text_clean.append(clean(df.loc[i]['Text_Data']))

transaction no is unresolved sorry to inform that there has been only a partial payment of amount gbp you are definitely going to receive the rest at a later point of time warm regards
order for new cheque book good morning i want to place an order for an multicity cheque book for account no having atleat leaves kindly send the same to the address mentioned in my account records
required money acquired transaction is in process hello this is to inform you that i have received the amount you transferred to my account and now it is currently in process
asking for the details for transaction i request you to kindly send the status of my transaction with id thanks and regards
partial payment for transaction hello greetings for the day status of transaction usd for account is pending i would be grateful if you could tell me the cause thanks a lot
payment done and transaction settled greetings i wanted to let you know that i have acknowledged the payment for transaction in response to your e

imploring update on transaction can you please tell me the amount transferred through transaction id thanks
urgent transaction ceased this is to notify you that my transaction has failed please reply to me with the cause as soon as possible
transaction stalled and payment not received this is in response to your email stating that my transaction having id has failed but no reason was mentioned can you please tell me what did i do wrong so that i can create a new one without any errors waiting for your reply thank you in advance
how to change pin no of atm card please reply this is very urgent i have to change the pin no for atm card associated with account no since it was stolen from me last night or suggest some other way to keep my money secure
seeking update on the status of transaction i need details of urgently please provide the same on priority basis
concluded the transaction greetings i wanted to let you know that i have acknowledged the payment for transaction in response to y

seeking update on the status of transaction kindly provide a status update on at the earliest
add another contact no to my bank account kindly add the contact no to my bank account and set it as my default no to receive all alerts
soliciting information for id hey i would be really grateful if you could tell me the details of account no thanks
request to send details of transaction it would be very kind of you to provide a status update on at the earliest hoping for a reply soon
failure of transaction greetings for the day i checked my inbox and found your email stating the failure of my transaction please help me understand why has failed need funds urgently patiently waiting for you reply thanks a lot
finalized transaction of id glad to let you know that i got the desired payment in reference to transaction i am very happy with your services and will definitely recommend your company to my friends and acquaintances warm regards
transaction no is unresolved hey sincere apologies for t

transaction no is unresolved since my transaction is still pending i wanted to know if there is a problem with the paperwork from my side please let me know at the earliest thanks and regards
partially paid the required amount for transaction hey sincere apologies for transferring a fraction of the amount we agreed on please be assured that the rest is being transferred as of now thanks a lot for understanding
pin number change hey i think i lost my atm card today so can you please block my account or tell me a way to change my pin no i urgently need your reply thanks in advance
soliciting information for id can you please tell me the amount transferred through transaction id thanks
required money acquired transaction is in process sincere greetings i am glad to tell you that i have finally been granted the pre approved amount through transaction it is now being processed i am grateful for your support warm regards
payment is pending for transaction hello greetings for the day status o

incomplete transaction since my transaction is still pending i wanted to know if there is a problem with the paperwork from my side please let me know at the earliest thanks and regards
handling the transaction after payment so happy to acknowledge the received payment for transaction it is finally in process kind regards
payment done and transaction settled i deeply appreciate your quick service as i have received the pre approved loan amount of inr
transaction is complete to whom it may concern i have successfully received payment for the transaction i am grateful for your cooperation thank you so much and regards
add one more contact no to account kindly add the contact no to my bank account
fulfilled transaction having id greetings i wanted to let you know that i have acknowledged the payment for transaction in response to your email which confirmed the finalized status of my transaction thanks and regards
incomplete transaction the transaction is taking too long to complete i woul

...

transaction having id has stopped help hey i see my transaction with id has failed i think i did everything right please look into this and reply with the reason urgently thank you and regards
failure of transaction greetings for the day i checked my inbox and found your email stating the failure of my transaction please help me understand why has failed need funds urgently patiently waiting for you reply thanks a lot
remaining amount for transaction to be paid later the transaction is taking too long to complete i would request you to kindly guide me through the further steps to be taken in order to complete the transaction
how to change pin no of atm card i need a new cheque book of leaves kindly send the same to my current address which is present in the details of my account
transaction having id has stopped help this is in response to your email stating that my transaction having id has failed but no reason was mentioned can you please tell me what did i do wrong so that i can cre

sorted out the transaction with id greetings i wanted to let you know that i have acknowledged the payment for transaction in response to your email which confirmed the finalized status of my transaction thanks and regards
asking for the details for transaction please provide a status update on at the earliest
abrupt closure of transaction with id this is in response to your email stating that my transaction having id has failed but no reason was mentioned can you please tell me what did i do wrong so that i can create a new one without any errors waiting for your reply thank you in advance
handling the transaction after payment this is in response to your email notifying about the transferred amount i am acknowledging it and informing you that it is now in process
handling the transaction after payment acknowledging the received payment for transaction it is finally in process kind regards
incomplete transaction i regret to inform you the i could only pay the partial amount of eur i w

sorted out the transaction with id hello sincere greetings for the day i would like to inform you that my transaction has completed thank you so much for your support looking forward to working more with you in the future regards
change address for account no i need a new cheque book of leaves kindly send the same to my current address which is present in the details of my account
add one more contact no to account this is to inform you that i am not able to withdraw money from my account please help
urgent transaction ceased greetings for the day i checked my inbox and found your email stating the failure of my transaction please help me understand why has failed need funds urgently patiently waiting for you reply thanks a lot
abrupt closure of transaction with id greetings for the day i checked my inbox and found your email stating the failure of my transaction please help me understand why has failed need funds urgently patiently waiting for you reply thanks a lot
change address for

soliciting information for id can you please tell me the amount transferred through transaction id thanks
payment outstanding for transaction there has been only a partial payment of amount eur assuring you that the rest will be paid later warm regards
change address for account no i need a new cheque book of leaves kindly send the same to my current address which is present in the details of my account
why cant i withdraw money with my atm card this is to inform you that i am not able to withdraw money from my account please help
payment is pending for transaction the transaction is taking too long to complete i would request you to kindly guide me through the further steps to be taken in order to complete the transaction
transaction no is unresolved hello greetings for the day status of transaction inr for account is pending i would be grateful if you could tell me the cause thanks a lot
want to block account kindly add the contact no to my bank account
dealing with the transaction t

upgrade to an account with more benefits one of your employees suggested me to upgrade my account since it has more benefits so i would like to confirm from you the perks that he listed thanks in advance
imploring update on transaction hey i would like to informed about the details of account no thanks
why has my transaction stopped i have been your regular client and have followed the procedure for creating a transaction correctly still i received an email saying that the transaction has failed i would like to know the reasoning my transaction id is regards
handling the transaction after payment this is in response to your email notifying about the transferred amount i am acknowledging it and informing you that it is now in process
request to send details of transaction can you please tell me the amount transferred through transaction id thanks
payment received for transaction and now processing grateful to you for transferring the payment to my account yesterday i would like to infor

In [119]:
df['Text_Data'] = text_clean

In [120]:
df.head()

,Subject,Body,Class,Text_Data
0,Transaction no. 072558 is unresolved.,Sorry to inform that there has been only a par...,Pending,transaction unresolved sorry inform partial pa...
1,Order for new Cheque book,"Good morning, I want to place an order for an ...",General,order new cheque book good morning want place ...
2,Required money acquired. Transaction 847047 is...,Hello! This is to inform you that I have recei...,Processing,require money acquire transaction process info...
3,Asking for the details for transaction 746078,I request you to kindly send the status of my ...,Request,ask detail transaction request kindly send sta...
4,Partial payment for transaction 535918,Hello!! Greetings for the day. Status of trans...,Pending,partial payment transaction day status transac...


In [121]:
df['Text_Data'][0]

'transaction unresolved sorry inform partial payment definitely go receive rest later point time warm'

In [122]:
# from sklearn.preprocessing import OneHotEncoder 
# le = OneHotEncoder()
# classes = df['Class'].values

# classes = classes.reshape(-1, 1)
# labels = le.fit_transform(classes)
# labels = labels.toarray()

In [123]:
labels = list(df['Class'].values)

In [124]:
df = df.drop(['Subject', 'Body'], axis=1)

In [125]:
# def stream_docs(path, path2):
#     with open(path, 'r') as csv1, open(path2,'r') as csv2:
#         next(csv1)
#         next(csv2)
#         for linex,liney in zip(csv1,csv2):
#             data=linex.split(",")
# #             print(data)
#             text=[]
#             text.append(data[2])
#             text.append(data[3])
#             label=liney[:-1]
# #             print(label)
#             yield text, label

# # next(stream_docs(path='./emaildataset.csv',path2='./classes.csv'))

In [126]:
pd.DataFrame(data = df.values).to_csv("emaildataset_core.csv",index = False, header = ['Class', 'Text_Data'])

In [127]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            data=line.split(",")
            text, label = data[1][:-1], data[0]
            yield text, label

In [128]:
next(stream_docs(path='./emaildataset_core.csv'))

('transaction unresolved sorry inform partial payment definitely go receive rest later point time warm',
 'Pending')

In [129]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    for _ in range(size):
        text, label = next(doc_stream)
        docs.append(text)
        y.append(label)
    return docs, y

In [130]:
from sklearn.feature_extraction.text import HashingVectorizer
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=clean)

In [131]:
from sklearn.linear_model import SGDClassifier
# SGDClassifier?
clf = SGDClassifier(max_iter=5)

doc_stream = stream_docs(path='./emaildataset_core.csv')

In [132]:
import pyprind
pbar = pyprind.ProgBar(90)

# classes = np.array([0, 1])
for _ in range(90):
    X_train, y_train = get_minibatch(doc_stream, size=10)
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=labels)
    pbar.update()

transaction unresolved sorry inform partial payment definitely go receive rest later point time warm
order new cheque book good morning want place order multicity cheque book have atleat leave kindly send address mention record
require money acquire transaction process inform receive transfer currently process
ask detail transaction request kindly send status transaction would
partial payment transaction day status transaction pende grateful tell cause lot
payment transaction settle want let know acknowledge payment transaction response email confirm finalized status transaction
failure transaction notify transaction fail reply cause soon possible
send step activate online banking hey transfer fund different bank urgently lockdown option online banking appreciate send instruction advance
incomplete transaction regret inform pay partial definitely pay remain soon possible thank supreme patience understanding
failure transaction hey transaction would fail think right look reply reason ur

/home/aheli/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


pending transaction reach soon transaction pende want know problem paperwork let know early
handle transaction payment sincere glad tell finally grant pre approve transaction process grateful support warm
upgrade benefit day branch have recently see advertisement state premium customer avail benefit decide upgrade guide step take order grateful reply day thank
process transaction have would acknowledge receive payment transaction finally process appreciative effort kind
issue new cheque book warm recently exhaust cheque book like order new cheque preference multicity cheque send early opportunity expect hear soon
pending transaction reach soon regret inform pay partial definitely pay remain soon possible thank understanding
transaction have would stop help regular client follow procedure create transaction correctly receive email say transaction fail like know reasoning transaction would
change address hey s time want change address mention record recently move previous place new addre

0% [#                             ] 100% | ETA: 00:00:27

want block hey think lose atm card today block tell way change pin urgently need reply advance
implore update transaction tell transfer transaction would
urgent transaction cease notify transaction fail reply cause soon possible
transaction stall payment not receive response email state transaction have would fail reason mention tell wrong create new error wait reply thank advance
change pin atm card reply urgent change pin atm card associate steal night suggest way money secure
seek update status transaction need detail urgently provide priority basis
conclude transaction want let know acknowledge payment transaction response email confirm finalized status transaction
payment outstanding transaction regretfully inform pay partial definitely pay remain soon possible thank understanding
fulfilled transaction have would deeply appreciate quick service receive pre approve loan
fulfilled transaction have would concern successfully receive payment transaction grateful cooperation thank
inco

...

0% [############################  ] 100% | ETA: 00:00:01

add contact inform not able withdraw money help
seek update status transaction kindly reply early transaction advance
payment receive transaction process thank transfer payment yesterday appreciate quick response like inform currently process
sort transaction would want let know acknowledge payment transaction response email confirm finalized status transaction
transaction complete concern successfully receive payment transaction grateful cooperation thank
deal transaction sincere glad tell finally grant pre approve transaction process grateful support warm
implore update transaction urgently require detail reply asap
failure transaction day check inbox find email state failure transaction help understand fail need fund urgently patiently wait reply lot
seek update status transaction provide status update early
add contact hear lot bank like open citibank like know document require kindly reply step take eagerly wait reply
payment pende transaction transaction take long complete reques

0% [############################# ] 100% | ETA: 00:00:00

incomplete transaction transaction take long complete request kindly guide step take order complete transaction
sort transaction would want let know acknowledge payment transaction response email confirm finalized status transaction
deal transaction sincere glad tell finally grant pre approve transaction process grateful support warm
partial payment transaction partial payment assure rest pay later warm
new cheque book kindly add contact bank
failure transaction regular client follow procedure create transaction correctly year receive email say transaction fail annoyed like know reason transaction would
upgrade benefit kindly add contact bank
failure transaction notify transaction fail reply cause soon possible
transaction fail day check inbox find email state failure transaction help understand fail need fund urgently patiently wait reply lot
impetrate detail would respond quickly highly appreciate send exact transfer yesterday
ask detail transaction tell transfer transaction would
ha

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:26


In [133]:
X_test, y_test = get_minibatch(doc_stream, size=51)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

implore update transaction request kindly send status transaction would
transaction stall payment not receive response email state transaction have would fail reason mention tell wrong create new error wait reply thank advance
transaction complete concern fortunately receive payment transaction thank cooperation
seek update status transaction urgently require detail early response expect
conclude transaction sincere day like inform transaction complete thank support look forward work future
pending transaction reach soon regret inform pay partial definitely pay remain soon possible thank understanding
incapable take money inform not able withdraw money help
abrupt closure transaction would regular client follow procedure create transaction correctly receive email say transaction fail like know reasoning transaction would
partially pay require transaction partial payment remain transfer appreciate patience
transaction stall payment not receive transaction not progress anymore fail not u

In [134]:
clf = clf.partial_fit(X_test, y_test)

## Serialize a Python object structure

In [136]:
import joblib
import os
if not os.path.exists('./pkl_objects'):
    os.mkdir('./pkl_objects')
    
joblib.dump(vect, './pkl_objects/vectorizer.pkl')
joblib.dump(clf, './pkl_objects/clf.pkl')

['./pkl_objects/clf.pkl']

In [140]:
%%writefile tokenizer.py
import re
import spacy
import en_core_web_sm

nlp_ = en_core_web_sm.load()
nlp = spacy.load('en')
my_stop = ["'d", "'ll", "'m", "'re", "'s", "'ve",'a','cc','subject','http', 'gbp', 'usd', 'eur', 'inr', 'cad', 'thanks', "acc", "id", 'account', 'regards', 'hi', 'hello', 'thank you', 'greetings', 'about','above', 'across','after','afterwards','against','alone','along','already','also','although','always','am','among', 'amongst','amount','an','and','another','any','anyhow','anyone','anything','anyway','anywhere','are','around','as', 'at','back','be','became','because','become','becomes','becoming','been','before','beforehand','behind','being','below', 'beside','besides','between','beyond','both','bottom','but','by','ca','call','can','could','did', 'do', 'does', 'doing', 'done', 'down', 'due', 'during', 'each', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had', 'has', 'have', 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'it', 'its', 'itself', 'just', 'keep', 'last', 'latter', 'latterly', 'least', 'less', 'made', 'make', 'many', 'may', 'me', 'meanwhile', 'might', 'mine', 'more', 'moreover', 'most', 'mostly', 'move', 'much', 'must', 'my', 'myself', 'name', 'namely', 'neither', 'never', 'nevertheless', 'next', 'nine', 'no', 'nobody', 'now', 'nowhere', 'of', 'off', 'often', 'on', 'once', 'one', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'part', 'per', 'perhaps', 'please', 'put', 'quite', 'rather', 're', 'really', 'regarding', 'same', 'say', 'see', 'seem', 'seemed', 'seeming', 'seems', 'serious', 'several', 'she', 'should', 'show', 'side', 'since', 'six', 'sixty', 'so', 'some', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhere', 'still', 'such', 'take', 'ten', 'than', 'that', 'the', 'their', 'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'third', 'this', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'top', 'toward', 'towards', 'twelve', 'twenty', 'two', 'under', 'unless', 'until', 'up', 'upon', 'us', 'used', 'using', 'various', 'very', 'via', 'was', 'we', 'well', 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with', 'within', 'would', 'yet', 'you', 'your', 'yours', 'yourself', 'yourselves', '‘d', '‘ll', '‘m', '‘re', '‘s', '‘ve', '’d', '’ll', '’m', '’re', '’s', '’ve']

def clean(text):    
    text=text.rstrip()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = " ".join([i for i in text.lower().split()])
#     print(text)
    
    doc = nlp(text)
    
    normalized = " ".join(token.lemma_ for token in doc if token not in my_stop)
    
    doc = " ".join(token.orth_ for token in nlp(normalized) if not token.is_punct | token.is_space)
    return doc

Overwriting tokenizer.py


In [142]:
from tokenizer import clean
joblib.dump(clean, './pkl_objects/clean.pkl')

['./pkl_objects/clean.pkl']

In [143]:
# Now, let us restart this IPython notebook and check if the we can load our serialized objects:

import joblib
tokenizer = joblib.load('./pkl_objects/clean.pkl')
vect = joblib.load('./pkl_objects/vectorizer.pkl')
clf = joblib.load('./pkl_objects/clf.pkl')
# After loading the tokenizer, HashingVectorizer, and the tranined logistic regression model, we can use it to make predictions on new data, which can be useful
# ,if we'd want to embed our classifier into a web application

In [144]:
example = ['I did not like this movie']
X = vect.transform(example)
clf.predict(X)

i did not like this movie


NameError: name 'nlp' is not defined

In [ ]:
example = ['I loved this movie']
X = vect.transform(example)
clf.predict(X)

In [45]:
# doc_stream = stream_docs(path='./emaildataset.csv',path2='./classes.csv')

In [21]:
# vectorizer = HashingVectorizer(decode_error='ignore', n_features=2 ** 18, alternate_sign=False)

In [22]:
# partial_fit_classifiers = {
#     'SGD': SGDClassifier(max_iter=5),
#     'Perceptron': Perceptron(),
#     'NB Multinomial': MultinomialNB(alpha=0.01),
#     'Passive-Aggressive': PassiveAggressiveClassifier(),
# }

## Implementing K-Means++

In [83]:
# parse commandline arguments
op = OptionParser()
op.add_option("--lsa",
              dest="n_components", type="int",
              help="Preprocess documents with latent semantic analysis.")
op.add_option("--no-minibatch",
              action="store_false", dest="minibatch", default=True,
              help="Use ordinary k-means algorithm (in batch mode).")
op.add_option("--no-idf",
              action="store_false", dest="use_idf", default=True,
              help="Disable Inverse Document Frequency feature weighting.")
op.add_option("--use-hashing",
              action="store_true", default=False,
              help="Use a hashing feature vectorizer")
op.add_option("--n-features", type=int, default=100000,
              help="Maximum number of features (dimensions)"
                   " to extract from text.")
op.add_option("--verbose",
              action="store_true", dest="verbose", default=False,
              help="Print progress reports inside k-means algorithm.")

print(__doc__)
op.print_help()

Automatically created module for IPython interactive environment
Usage: ipykernel_launcher.py [options]

Options:
  -h, --help            show this help message and exit
  --lsa=N_COMPONENTS    Preprocess documents with latent semantic analysis.
  --no-minibatch        Use ordinary k-means algorithm (in batch mode).
  --no-idf              Disable Inverse Document Frequency feature weighting.
  --use-hashing         Use a hashing feature vectorizer
  --n-features=N_FEATURES
                        Maximum number of features (dimensions) to extract
                        from text.
  --verbose             Print progress reports inside k-means algorithm.


In [100]:
df.head()

,From,To,Subject,Body,TransactionID,Payment Date,Class,Text_Data
0,Spike@DEUTSCHEBANK.com,Shaniece@CitiBankPune.com,Transaction no. 072558 is unresolved.,Sorry to inform that there has been only a par...,72558,04-02-2020,Pending,transaction unresolved sorry inform partial pa...
1,Stacy@HDFC.com,Rohan@CitiBankNewYork.com,Order for new Cheque book,"Good morning, I want to place an order for an ...",212096,29-05-2020,General,order new cheque book good morning want place ...
2,Zachary@HDFC.com,Rishabh@CitiBankPune.com,Required money acquired. Transaction 847047 is...,Hello! This is to inform you that I have recei...,847047,26-01-2020,Processing,require money acquire transaction process info...
3,Stacy@SBI.com,Shai@CitiBankHongKong.com,Asking for the details for transaction 746078,I request you to kindly send the status of my ...,746078,17-06-2019,Request,ask detail transaction request kindly send sta...
4,Angela@HDFC.com,Dipesh@CitiBankSingapore.com,Partial payment for transaction 535918,Hello!! Greetings for the day. Status of trans...,535918,18-02-2020,Pending,partial payment transaction day status transac...


In [106]:
print("Extracting features from the training dataset using a sparse vectorizer")

t0 = time()

if opts.use_hashing:
    if opts.use_idf:
        # Perform an IDF normalization on the output of HashingVectorizer
        hasher = HashingVectorizer(n_features=opts.n_features, stop_words=my_stop, alternate_sign=False, norm=None)
        vectorizer = make_pipeline(hasher, TfidfTransformer())
    
    else:
        vectorizer=HashingVectorizer(n_features=opts.n_features, stop_words=my_stop, 
                                     alternate_sign=False, norm='l2')

else:       
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=opts.n_features,
                                 min_df=2, stop_words=my_stop, use_idf=opts.use_idf)
    

X = vectorizer.fit_transform(text_clean)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

if opts.n_components:
    print("Performing dimensionality reduction using LSA")
    t0 = time()
    # Vectorizer results are normalized, which makes KMeans behave as
    # spherical k-means for better results. Since LSA/SVD results are 
    # not normalized, we have to redo the normalization.
    
    svd = TruncatedSVD(opts.n_components)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)

    X = lsa.fit_transform(X)

    print("done in %fs" % (time() - t0))

    explained_variance = svd.explained_variance_ratio_.sum()
    print("Explained variance of the SVD step: {}%".format(
        int(explained_variance * 100)))

    print()

Extracting features from the training dataset using a sparse vectorizer
done in 0.041258s
n_samples: 951, n_features: 212



/home/aheli/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 'thank', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [108]:
# Do the actual clustering

if opts.minibatch:
    km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', n_init=1,
                         init_size=5, batch_size=5, verbose=opts.verbose)
else:
    km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1,
                verbose=opts.verbose)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, km.labels_, sample_size=1000))

print()


if not opts.use_hashing:
    print("Top terms per cluster:")

    if opts.n_components:
        original_space_centroids = svd.inverse_transform(km.cluster_centers_)
        order_centroids = original_space_centroids.argsort()[:, ::-1]
    else:
        order_centroids = km.cluster_centers_.argsort()[:, ::-1]

    terms = vectorizer.get_feature_names()
    for i in range(true_k):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()

Clustering sparse data with MiniBatchKMeans(batch_size=5, compute_labels=True, init='k-means++',
        init_size=5, max_iter=100, max_no_improvement=10, n_clusters=6,
        n_init=1, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=False)
done in 0.139s

Homogeneity: 0.341
Completeness: 0.569
V-measure: 0.426
Adjusted Rand-Index: 0.125
Silhouette Coefficient: 0.092

Top terms per cluster:
Cluster 0: process currently transfer inform payment receive acknowledge response finally kind
Cluster 1: pay partial rest understanding assure remain later payment regret soon
Cluster 2: fail kindly detail reply complete request tell know status update
Cluster 3: finalized confirm want let acknowledge status know response email payment
Cluster 4: wrong mention error reason state new create advance wait fail
Cluster 5: priority basis detail provide urgently need update request send require


/home/aheli/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1562: RuntimeWarning: init_size=5 should be larger than k=6. Setting it to 3*k
  init_size=init_size)
